In [1]:
from utils.data_handling import *
from utils.sensor_optimisation_gpy import *
from parameters import *

import matplotlib.pyplot as plt

np.random.seed(101)

In [2]:
parameters['i_end'] = 100
parameters

{'i_start': 0,
 'i_end': 100,
 'crop': ((-10, 10), (-10, 10), (0, 50)),
 'field_name': 'TracerBackground',
 'cov_method': 'sample'}

In [3]:
loaded = initial_load_data(parameters, recompute=False)
ref_vtu, data_df, loc_df, time_df = loaded

  1%|          | 1/101 [00:00<00:22,  4.41it/s]

### Loading preprocessed files
==> Import vtu files from 0 to 100
==> Cropping vtu files to ((-10, 10), (-10, 10), (0, 50))


100%|██████████| 101/101 [00:23<00:00,  4.28it/s]

Number of Locations after cropping :  1129
==> Saving to : ../data/temp_data/cache_0_100_((-10, 10), (-10, 10), (0, 50))/loc_TracerBackground.pkl
==> Saving to : ../data/temp_data/cache_0_100_((-10, 10), (-10, 10), (0, 50))/time_TracerBackground.pkl
==> Saving to : ../data/temp_data/cache_0_100_((-10, 10), (-10, 10), (0, 50))/data_TracerBackground.pkl


In [5]:
# Data for the regression
dim = 3
t = 100
X = loc_df.values #[I,:dim]
Y = data_df.values[:,t].reshape(-1,1) # [I,t].reshape(-1,1)
Y.shape

(1129, 1)

Setting the kernel

In [6]:
kernel = GPy.kern.Matern52(dim,ARD=False) + GPy.kern.White(dim)
m_train = GPy.models.GPRegression(X,Y,kernel)
m_train.optimize(messages=True,max_f_eval = 1000)


## Sensor Optimisation

Define the Sets for the optimisation

In [7]:
n_V = X.shape[0]
sets = define_sets(n_V)

Number of sensors to place : 

In [8]:
k = 10

### Lazy Sensor Optimisation

In [9]:
sensor_loc_optimisation_lazy(k, sets, X, Y, kernel)

100%|██████████| 10/10 [05:46<00:00, 26.69s/it]


array([ 964,  280,  431,  386,  397,  405,  482,  523, 1044,  965])

### Naive Sensor Optimisation

In [10]:
k = 10
sensor_loc_optimisation_naive(k, sets, X, Y, kernel)

100%|██████████| 10/10 [01:26<00:00,  8.70s/it]


array([106,  66,  35, 181,  44,  89,  59,  72,  18,  83])